In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Nygaard TDM

In [2]:
U_0 = 10
z_0 = 0.001
D = 100
z_H = 100
C_T = 0.8
s_x = 10
s_y = 10
s_d = s_x*D
s_c = s_y*D
kappa = 0.4

In [3]:
def calculate_wind_speed(x_1, x_2, x_3, z_H, D, C_T, s_d, s_c):

    c_t = (np.pi * C_T * D**2) / (8 * s_d * s_c)
    print('c_t', c_t)

    nu_star = 28/10 * np.sqrt(c_t)
    
    beta =  nu_star / (1 + nu_star)
    
    z_0WF = z_H * (1 + D / (2 * z_H))**beta * np.exp(-kappa * (c_t + (kappa / np.log((z_H / z_0) * (1 - D / (2 * z_H))**beta))**2)**-0.5)

    x = np.linspace(x_1, x_3-1, int(x_3-x_1))

    h_1 = np.full_like(x, np.nan, dtype=float)
    h_2 = np.full_like(x, np.nan, dtype=float)

    alpha = 1.6

    h_1[x >= x_1] = z_H + ((x[x >= x_1] - x_1) / z_0WF)**(alpha) * z_0WF
    h_2[x >= x_2] =   0 + ((x[x >= x_2] - x_2) / z_0WF)**(alpha) * z_0WF

    U = np.full_like(x, np.nan, dtype=float)
    U[0] = U_0

    idx1 = h_1 >= z_H
    U[idx1] = U_0 * (np.log(h_1[idx1] / z_0) * np.log(z_H / z_0WF)) / (np.log(h_1[idx1] / z_0WF) * np.log(z_H / z_0))

    idx2 = (x >= x_2) & (h_2 >= z_H)
    U[idx2] = U_0 * (np.log(h_1[idx2] / z_0) * np.log(h_2[idx2] / z_0WF)) / (np.log(h_1[idx2] / z_0WF) * np.log(h_2[idx2] / z_0))

    for i in range(1, len(U)):
        if U[i] > U_0:
            U[i] = U_0
        elif U[i] < 0:
            U[i] = 0

    return x, U, c_t

# Alex's firkanter :)

In [4]:
d = 1400 * D

l_farm = int(12 * s_d)
w_farm = int(12 * s_c + (2*D/2))
h_farm = D

k_w = 0.005

x, u_tdm, c_t = calculate_wind_speed(0, l_farm, d, z_H, D, C_T, s_d, s_c)

A_0 = (w_farm * h_farm) + (2 * w_farm * k_w * l_farm) + (2 * h_farm * k_w * l_farm) + (4 * (k_w * l_farm)**2)

u_s = (U_0 - u_tdm[-1]) / U_0
print('U', u_tdm[-1])
#print('u_s', u_s)

c_t 0.003141592653589793
U 9.985943382748312


In [5]:
kx = (- h_farm/4 - w_farm/4 + (np.sqrt(h_farm**2 + 2*A_0 + w_farm**2 - 4*A_0 + 4*(A_0 * (C_T / (2*(u_s - u_s**2))))))/4)/d
print('kx:', round(kx,5), '[m/m]')

kx: 0.07932 [m/m]


# Save to CSV

In [6]:
k = int(kx*d)
y_meas = np.linspace(0 - k, w_farm + k, w_farm+(2*k)+1)
U = np.full_like(y_meas, u_tdm[-1])

In [7]:
data = pd.DataFrame({
    "y_wake": y_meas / D,
    "U_norm": U / U_0,
    "kx": kx,
})
data.to_csv(f"../ArbitraryLayouts/ValidationData/{int(d / D)}D.csv", index=False)